In [1]:
import json
import foursquareWithHosts
from datetime import datetime
from sys import stdout
import requests
with open('keys/appCredentialsSudo.json') as credentialsFile:    
    credentials = json.load(credentialsFile)
with open('keys/accessTokensSudo.json') as tokensFile:    
    accessTokens = json.load(tokensFile)
requests.packages.urllib3.disable_warnings()

In [2]:
clients = []
users = []
for accessToken in accessTokens:
    client = foursquareWithHosts.Foursquare(client_id=credentials['client_id'], client_secret=credentials['client_secret'], redirect_uri=credentials['redirect_uri'])
    client.set_access_token(accessToken)
    user = client.users()
    clients.append(client)
    users.append(user)

Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi


# Check and add new friends

In [3]:
def progress(i, n):
    stdout.write("\r%f%%" % (i*100/float(n)))
    stdout.flush()
    if i == n-1:
        stdout.write("\r100%")
        print("\r\n")

In [4]:
def getMongoDBInstance():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.habidatum
    return db

In [5]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
db = getMongoDBInstance()

In [6]:
def fillSourceDataFrames(folderName):
    checkinsDF = pd.read_csv('data/'+folderName+'checkins.tsv', sep='\t',encoding='utf-8')
    checkpointsDF = pd.read_csv('data/'+folderName+'checkPoints.tsv', sep='\t',encoding='utf-8')
    usersDF = pd.read_csv('data/'+folderName+'users.tsv', sep='\t',encoding='utf-8')
    return (checkinsDF, checkpointsDF, usersDF)

In [7]:
with open('data/pendingFriendsIds.json') as oldPendingFriendsFile:    
    oldPendingFriends = json.load(oldPendingFriendsFile)

In [8]:
#Getting old  pending users
def getPendingFriends(db):
    existingPendingFriends = oldPendingFriends
    existingPendingFriendsCoursor = db.pendingUsers.find()
    for pendingFriend in existingPendingFriendsCoursor:
        existingPendingFriends.append(int(pendingFriend['requestedUser']['user']['id']))
    return existingPendingFriends

In [9]:
def getUnfriendedFriendsForUser(db, userIndex):
    existingPendingFriends = []
    existingPendingFriendsCoursor = db.unfriendedUsers.find({"unfriendingUser":userIndex})
    for pendingFriend in existingPendingFriendsCoursor:
        existingPendingFriends.append(int(pendingFriend['unfriendedUser']))
    return existingPendingFriends

In [10]:
def getPendedFriendsForUser(db, userIndex):
    existingPendingFriends = []
    existingPendingFriendsCoursor = db.pendingUsers.find({"requestingUser":userIndex})
    for pendingFriend in existingPendingFriendsCoursor:
        existingPendingFriends.append(int(pendingFriend['requestedUser']['user']['id']))
    return existingPendingFriends

In [11]:
def getUsersToFriend(usersDF, existingPendingFriends):
    return usersDF[~usersDF.userId.isin(existingPendingFriends)].userId.tolist()

In [12]:
def countUsersToFriend(usersToFriend):
    usersToFriendCount = len(usersToFriend)
    if usersToFriendCount:
        print('{} users to friend').format(usersToFriendCount)
    else:
        print('No new users to friend')
    return usersToFriendCount

In [17]:
def unfriendAllUsers( db, clients):
    for clientId, client in enumerate(clients):
        print ''
        print 'Client:', clientId
        pendedUsers = getPendedFriendsForUser(db, clientId)
        unfriendedUsers = getUnfriendedFriendsForUser(db, clientId)
        print ''
        print 'Pending: ', len(pendedUsers)
        print 'Already unfriended: ', len(unfriendedUsers)
        for index, userId in enumerate(pendedUsers[:-50]):
            progress(index, len(pendedUsers[:-50]))
            if userId not in unfriendedUsers:
                try:
                    clients[clientId].users.unfriend(userId)
                    db.unfriendedUsers.insert({'unfriendingUser':clientId,
                                                'unfriendedUser':userId,
                                                'unfriendedAt':datetime.now()})
                except Exception as e:
                    pass

In [ ]:
unfriendAllUsers(db, clients)

In [14]:
def friendUsers(usersToFriend, db, clients):
    pendingUsers = []
    for index, userId in enumerate(usersToFriend):
        progress(index,len(usersToFriend))
        currentUserId = index%len(clients)
        try:
            pendingUser = clients[currentUserId].users.request(userId)
            pendingUsers.append(pendingUser)
            db.pendingUsers.insert({'requestingUser':currentUserId,
                                        'requestedUser':pendingUser,
                                         'requestedAt':datetime.now()})
        except Exception as e:
            print ''
            print e
            print 'User',currentUserId, 'Can\'t friend'
            
            pendedUsers = getPendedFriendsForUser(db, currentUserId)
            unfriendedUsers = getUnfriendedFriendsForUser(db, currentUserId)
            for index, userId in enumerate(pendedUsers):
                progress(index, 400)
                if userId not in unfriendedUsers:
                    try:
                        clients[currentUserId].users.unfriend(userId)
                        db.unfriendedUsers.insert({'unfriendingUser':currentUserId,
                                                    'unfriendedUser':userId,
                                                     'unfriendedAt':datetime.now()})
                    except Exception as e:
                        pass
    return pendingUsers

In [15]:
def addNewUsersFromCheckins(folderName, clients, db):
    (checkinsDF, checkpointsDF, usersDF) = fillSourceDataFrames(folderName)
    existingPendingFriends = getPendingFriends(db)
    usersToFriend = getUsersToFriend(usersDF, existingPendingFriends)
    if countUsersToFriend(usersToFriend):
        friendUsers(usersToFriend, db, clients)

# Run the loop

In [ ]:
import schedule
import time

schedule.clear()
addNewUsersFromCheckins('July/', clients, db)
schedule.every(10).minutes.do(addNewUsersFromCheckins,'July/', clients, db)
schedule.every(24).hours.do(unfriendAllUsers,db, clients)

while True:
    schedule.run_pending()
    time.sleep(1)

3 users to friend
66.666667%

In [ ]:
addNewUsersFromCheckins('July/', clients, db)

In [26]:
existingPendingFriendsCoursor0 = db.pendingUsers.find({'requestingUser':4})

In [35]:
for index, userId in enumerate(oldPendingFriends[-500:]):
    removed = False
    progress(index, 500)
    for client in clients:
        if removed:
            continue
        try:
            client.users.unfriend(userId)
            removed = True
        except:
            pass

100%

